In [31]:
import glob
import os
import pickle
import sys
from pathlib import Path
import random
from tqdm import tqdm
import pyvips as pv
import openslide

# change these to your local paths
from slidl.slide import Slide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import albumentations as A

import time
import logging

from PytorchUNet.unet import UNet
#from PytorchUNet.utils.dataset import SlidlSegmentationDataset, visualizeSegmentationAugmentation
from torch.utils.data import DataLoader, random_split

In [32]:
# change to your local paths
analysis_dir_path = './slidl-tutorial'
train_path_dr=  './train'
val_path_dr =  './val'

#wsi_path = './wsi_data'
wsi_path_train = './wsi_data2/train'
wsi_path_val = './wsi_data2/val'
final_slide_path='./final_tiles'

slidl_slide_dir_path = os.path.join(analysis_dir_path, 'slidl_slides')
train_tiles_dir_path = os.path.join(slidl_slide_dir_path, 'train_tiles')
val_tiles_dir_path = os.path.join(slidl_slide_dir_path, 'val_tiles')


# creat the different paths 
os.makedirs(analysis_dir_path, exist_ok=True)
os.makedirs(train_path_dr, exist_ok=True)
os.makedirs(val_path_dr, exist_ok=True)
os.makedirs(slidl_slide_dir_path, exist_ok=True)
os.makedirs(train_tiles_dir_path , exist_ok=True)
os.makedirs(val_tiles_dir_path , exist_ok=True)
# put the images into a list 
train_cases = os.listdir(wsi_path_train)
val_cases = os.listdir(wsi_path_val)



#train_path = [os.path.join(wsi_path_train,  train_case) for train_case in train_cases]
#val_path = [os.path.join(wsi_path_val, val_cases) for val_cases in val_cases]

In [33]:
# Im this part we remove all the ".jpg" extension 
wsi_path=os.listdir('/Users/eudari01/GNNs_Vs_CNNs/split_images/has_CIN_HE_trainsetCV_fold_0')
H_E_images=[x.split('.jpg')[0] for x in wsi_path]
print(H_E_images)

['MB-0310', 'MB-0028', 'MB-0106', 'MB-0568', 'MB-0164', 'MB-0571', 'MB-0524', 'MB-0167', 'MB-0311', 'MB-0406', 'MB-0313', 'MB-0224', 'MB-0395', 'MB-0349', 'MB-0542', 'MB-0569', 'MB-0131', 'MB-0149', 'MB-0287', 'MB-0192', 'MB-0414', 'MB-0413', 'MB-0336', 'MB-0541', 'MB-0295', 'MB-0138', 'MB-0123', 'MB-0364', 'MB-0363', 'MB-0197', 'MB-0362', 'MB-0194', 'MB-0383', 'MB-0394', 'MB-0386', 'MB-0346', 'MB-0143', 'MB-0544', 'MB-0354', 'MB-0877', 'MB-0162', 'MB-0328', 'MB-0380', 'MB-0151', 'MB-0128', 'MB-0353', 'MB-0195', 'MB-0177', 'MB-0325', 'MB-0358', 'MB-0317', 'MB-0152', 'MB-0536', 'MB-0120', 'MB-0315', 'MB-0232', 'MB-0306', 'MB-0060', 'MB-0204', 'MB-0341', 'MB-0344', 'MB-0144', 'MB-0356', 'MB-0122', 'MB-0302', 'MB-0401', 'MB-0529', 'MB-0316', 'MB-0180', 'MB-0172', 'MB-0112', 'MB-0320', 'MB-0321', 'MB-0171', 'MB-0384']


- JPEG images have just width x height pixels. There are no tiles and no pyramid levels.

- TIFF has many variations, but one is pyramidal and tiled, with JPEG compression for each tile. libvips can read and write these. We can use page=xxx to pick a pyramid level in pyvips.

in this part we do the conversion for all the images with page = 0.

In [34]:
    for eachFile in H_E_images:
  
        image = pv.Image.new_from_file( "/Users/eudari01/GNNs_Vs_CNNs/split_images/has_CIN_HE_trainsetCV_fold_0/" + eachFile +".jpg"  , access="sequential")
    

        image.write_to_file("./test2/" + eachFile+ ".tif" , tile=True, pyramid=True, compression="jpeg")
        image = pv.Image.new_from_file("./test2/"+ eachFile + ".tif" ,page=0)
        image.write_to_file("./test4/"+ eachFile +".tif", tile=True, pyramid=True, compression="jpeg")


In [35]:
H_E_images_tiff = os.listdir('./test4/')
H_E_images_tiff 

['MB-0306.tif',
 'MB-0364.tif',
 'MB-0346.tif',
 'MB-0138.tif',
 'MB-0363.tif',
 'MB-0536.tif',
 'MB-0144.tif',
 'MB-0131.tif',
 'MB-0386.tif',
 'MB-0877.tif',
 'MB-0395.tif',
 'MB-0541.tif',
 'MB-0317.tif',
 'MB-0568.tif',
 'MB-0028.tif',
 'MB-0358.tif',
 'MB-0394.tif',
 'MB-0106.tif',
 'MB-0344.tif',
 'MB-0401.tif',
 'MB-0112.tif',
 'MB-0302.tif',
 'MB-0569.tif',
 'MB-0321.tif',
 'MB-0336.tif',
 'MB-0120.tif',
 'MB-0197.tif',
 'MB-0167.tif',
 'MB-0542.tif',
 'MB-0295.tif',
 'MB-0413.tif',
 'MB-0571.tif',
 'MB-0349.tif',
 'MB-0152.tif',
 'MB-0194.tif',
 'MB-0406.tif',
 'MB-0287.tif',
 'MB-0122.tif',
 'MB-0414.tif',
 'MB-0143.tif',
 'MB-0320.tif',
 'MB-0384.tif',
 'MB-0149.tif',
 'MB-0325.tif',
 'MB-0316.tif',
 'MB-0180.tif',
 'MB-0204.tif',
 'MB-0362.tif',
 'MB-0195.tif',
 'MB-0192.tif',
 'MB-0060.tif',
 'MB-0311.tif',
 'MB-0151.tif',
 'MB-0123.tif',
 'MB-0341.tif',
 'MB-0313.tif',
 'MB-0171.tif',
 'MB-0524.tif',
 'MB-0544.tif',
 'MB-0162.tif',
 'MB-0310.tif',
 'MB-0328.tif',
 'MB-017

We can split the images in train set (70%), validtion set (15%) and test set (15%).

In [52]:
random.shuffle(H_E_images_tiff)
    
split_1 = int(0* len( H_E_images_tiff))
train_cases=  H_E_images_tiff[:split_1]

random.shuffle(H_E_images_tiff)

split_2 = int(0 * len(H_E_images_tiff))
val_cases=  H_E_images_tiff[:split_2]

random.shuffle(H_E_images_tiff)

split_3 = int(0 * len(H_E_images_tiff))
test_cases = H_E_images_tiff[:split_3]
  


In [42]:
len(train_cases)

75

In [43]:

train_path = [os.path.join('./test4/', train_case) for train_case in train_cases]
val_path = [os.path.join('./test4/', val_case) for val_case in val_cases]
#test_path = [os.path.join('./test4/', test_case) for test_case in test_cases]

In [44]:
train_path

['./test4/MB-0232.tif',
 './test4/MB-0336.tif',
 './test4/MB-0143.tif',
 './test4/MB-0151.tif',
 './test4/MB-0524.tif',
 './test4/MB-0568.tif',
 './test4/MB-0401.tif',
 './test4/MB-0112.tif',
 './test4/MB-0316.tif',
 './test4/MB-0123.tif',
 './test4/MB-0131.tif',
 './test4/MB-0320.tif',
 './test4/MB-0317.tif',
 './test4/MB-0172.tif',
 './test4/MB-0177.tif',
 './test4/MB-0353.tif',
 './test4/MB-0144.tif',
 './test4/MB-0325.tif',
 './test4/MB-0364.tif',
 './test4/MB-0060.tif',
 './test4/MB-0195.tif',
 './test4/MB-0180.tif',
 './test4/MB-0306.tif',
 './test4/MB-0380.tif',
 './test4/MB-0120.tif',
 './test4/MB-0152.tif',
 './test4/MB-0362.tif',
 './test4/MB-0384.tif',
 './test4/MB-0192.tif',
 './test4/MB-0536.tif',
 './test4/MB-0358.tif',
 './test4/MB-0414.tif',
 './test4/MB-0354.tif',
 './test4/MB-0128.tif',
 './test4/MB-0341.tif',
 './test4/MB-0328.tif',
 './test4/MB-0287.tif',
 './test4/MB-0197.tif',
 './test4/MB-0344.tif',
 './test4/MB-0310.tif',
 './test4/MB-0413.tif',
 './test4/MB-035

In [45]:
val_path

[]

First we'll define a SliDL Slide object for each whole-slide image (WSI). We can use SliDL to break the image up into tiles of the desired size and separate all tiles showing background and artifact from those showing actual tissue.

In [46]:
# For the train set 
start_time = time.time()

tile_size = 256# pixels

for wsi_path_train in train_path:
    case = Path(wsi_path_train).stem
    slidl_slide = Slide(wsi_path_train).setTileProperties(tileSize=tile_size)
    #slidl_slide.detectTissue() 
    slidl_slide.detectForeground()
    slidl_slide.save(folder=train_tiles_dir_path )
        

    
time_elapsed = time.time() - start_time
print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
Complete in 0m 51s


In [26]:
# For the validation set 
start_time = time.time()

tile_size = 256 # pixels

for wsi_path_val in val_path:
    case = Path(wsi_path_val).stem
    slidl_slide = Slide(wsi_path_val).setTileProperties(tileSize=tile_size)
    #slidl_slide.detectTissue() 
    slidl_slide.detectForeground()
    slidl_slide.save(folder=val_tiles_dir_path)
        

    
time_elapsed = time.time() - start_time
print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
tiffload
Complete in 0m 19s


In [12]:
#analysis_dir_path2='./slidl-tutorial/slidl_slides/val_tiles/Image.pml'
#for case in val_cases:
    #slidl_slide = Slide(os.path.join(analysis_dir_path2))
    #slidl_slide.visualizeThumbnail(folder=os.path.join(analysis_dir_path2))
    #slidl_slide.visualizeForeground('otsu', folder=os.path.join(analysis_dir_path2))
    #slidl_slide.visualizeForeground('triangle', folder=os.path.join(analysis_dir_path2))
    #slidl_slide.visualizeTissueDetection(folder=os.path.join(analysis_dir_path2))

In [47]:
analysis_dir_path2='./slidl-tutorial/slidl_slides/val_tiles'
analysis_dir_path3='./slidl-tutorial/slidl_slides/train_tiles'
val_cases_pml = os.listdir(val_tiles_dir_path)
train_cases_pml = os.listdir(train_tiles_dir_path)
final_slide_path='./final_tiles'

In [50]:
val_cases_pml

['MB-0232.pml',
 'MB-0203.pml',
 'MB-0353.pml',
 'MB-0143.pml',
 'MB-0414.pml',
 'MB-0536.pml',
 'MB-0123.pml',
 'MB-0349.pml',
 'MB-0112.pml',
 'MB-0371.pml',
 'MB-0126.pml',
 'MB-0368.pml',
 'MB-0325.pml',
 'MB-0257.pml',
 'MB-0140.pml',
 'MB-0385.pml',
 'MB-0316.pml',
 'MB-0131.pml',
 'MB-0370.pml',
 'MB-0356.pml',
 'MB-0122.pml',
 'MB-0393.pml',
 'MB-0362.pml',
 'MB-0229.pml',
 'MB-0382.pml',
 'MB-0406.pml',
 'MB-0313.pml',
 'MB-0420.pml',
 'MB-0352.pml',
 'MB-0108.pml']

In [49]:
train_cases_pml

['MB-0192.pml',
 'MB-0180.pml',
 'MB-0232.pml',
 'MB-0320.pml',
 'MB-0302.pml',
 'MB-0529.pml',
 'MB-0353.pml',
 'MB-0028.pml',
 'MB-0143.pml',
 'MB-0321.pml',
 'MB-0414.pml',
 'MB-0171.pml',
 'MB-0536.pml',
 'MB-0344.pml',
 'MB-0151.pml',
 'MB-0123.pml',
 'MB-0106.pml',
 'MB-0349.pml',
 'MB-0224.pml',
 'MB-0354.pml',
 'MB-0112.pml',
 'MB-0383.pml',
 'MB-0394.pml',
 'MB-0328.pml',
 'MB-0341.pml',
 'MB-0386.pml',
 'MB-0380.pml',
 'MB-0287.pml',
 'MB-0363.pml',
 'MB-0325.pml',
 'MB-0317.pml',
 'MB-0395.pml',
 'MB-0364.pml',
 'MB-0315.pml',
 'MB-0384.pml',
 'MB-0128.pml',
 'MB-0138.pml',
 'MB-0316.pml',
 'MB-0131.pml',
 'MB-0149.pml',
 'MB-0568.pml',
 'MB-0571.pml',
 'MB-0358.pml',
 'MB-0162.pml',
 'MB-0306.pml',
 'MB-0356.pml',
 'MB-0122.pml',
 'MB-0144.pml',
 'MB-0204.pml',
 'MB-0362.pml',
 'MB-0197.pml',
 'MB-0172.pml',
 'MB-0401.pml',
 'MB-0877.pml',
 'MB-0541.pml',
 'MB-0177.pml',
 'MB-0152.pml',
 'MB-0060.pml',
 'MB-0406.pml',
 'MB-0336.pml',
 'MB-0120.pml',
 'MB-0346.pml',
 'MB-056

We can extract all tiles

In [51]:
start_time = time.time()

global_channel_sums = np.zeros(3)
global_channel_squared_sums = np.zeros(3)
global_tile_count = 0

for case in train_cases_pml:
    
    slidl_slide = Slide(os.path.join(analysis_dir_path3,case))
    #suitable_tile_addresses = slidl_slide.suitableTileAddresses(tissueLevelThreshold=0.995, foregroundLevelThreshold=88)
    
    channel_data = slidl_slide.extractRandomUnannotatedTiles(train_path_dr ,numTilesToExtract= 1000, unannotatedClassName='train_metastasis', foregroundLevelThreshold=70)
                                     
    global_channel_sums = np.add(global_channel_sums, channel_data['channel_sums'])
    global_channel_squared_sums = np.add(global_channel_squared_sums, channel_data['channel_squared_sums'])
    global_tile_count = global_tile_count + channel_data['num_tiles']
        
time_elapsed = time.time() - start_time
print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

tiffload
Extracting 6 of 6 train_metastasis tiles...
tiffload
Extracting 3 of 3 train_metastasis tiles...
tiffload
Extracting 15 of 15 train_metastasis tiles...
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extracting 25 of 25 train_metastasis tiles...
tiffload
Extracting 5 of 5 train_metastasis tiles...
tiffload
Extracting 25 of 25 train_metastasis tiles...
tiffload
Extracting 17 of 17 train_metastasis tiles...
tiffload
Extracting 8 of 8 train_metastasis tiles...
tiffload
Extracting 22 of 22 train_metastasis tiles...
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extracting 5 of 5 train_metastasis tiles...
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extracting 4 of 4 train_metastasis tiles...
tiffload
Extracting 3 of 3 train_metastasis tiles...
tiffload
Extracting 3 of 3 train_metastasis tiles...
tiffload
Extracting 5 of 5 train_metastasis tiles...
tiffload
Extracting 4 of 4 train_metastasis tiles...
tiffload
Extracting 12 of 12 train_m

tiffload
Extracting 16 of 16 train_metastasis tiles...
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extracting 3 of 3 train_metastasis tiles...
tiffload
Extracting 9 of 9 train_metastasis tiles...
tiffload
Extracting 17 of 17 train_metastasis tiles...
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extracting 17 of 17 train_metastasis tiles...
tiffload
Extracting 8 of 8 train_metastasis tiles...
tiffload
Extracting 3 of 3 train_metastasis tiles...
tiffload
tiffload
Extracting 8 of 8 train_metastasis tiles...
tiffload
Extracting 5 of 5 train_metastasis tiles...
tiffload
Extracting 20 of 20 train_metastasis tiles...
tiffload
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extracting 3 of 3 train_metastasis tiles...
tiffload
Extracting 4 of 4 train_metastasis tiles...
tiffload
Extracting 4 of 4 train_metastasis tiles...
tiffload
Extracting 26 of 26 train_metastasis tiles...
tiffload
Extracting 1 of 1 train_metastasis tiles...
tiffload
Extractin

In [30]:
start_time = time.time()

global_channel_sums = np.zeros(3)
global_channel_squared_sums = np.zeros(3)
global_tile_count = 0

for case in val_cases_pml:
    slidl_slide = Slide(os.path.join(analysis_dir_path2,case))
    channel_data = slidl_slide.extractRandomUnannotatedTiles(val_path_dr,numTilesToExtract= 1000, unannotatedClassName='val_metastasis', foregroundLevelThreshold=70)
                                     
    global_channel_sums = np.add(global_channel_sums, channel_data['channel_sums'])
    global_channel_squared_sums = np.add(global_channel_squared_sums, channel_data['channel_squared_sums'])
    global_tile_count = global_tile_count + channel_data['num_tiles']
        
time_elapsed = time.time() - start_time
print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

tiffload
Extracting 15 of 15 val_metastasis tiles...
tiffload
Extracting 13 of 13 val_metastasis tiles...
tiffload
Extracting 25 of 25 val_metastasis tiles...
tiffload
Extracting 8 of 8 val_metastasis tiles...
tiffload
Extracting 1 of 1 val_metastasis tiles...
tiffload
Extracting 1 of 1 val_metastasis tiles...
tiffload
Extracting 3 of 3 val_metastasis tiles...
tiffload
Extracting 4 of 4 val_metastasis tiles...
tiffload
Extracting 25 of 25 val_metastasis tiles...
tiffload
Extracting 1 of 1 val_metastasis tiles...
tiffload
Extracting 20 of 20 val_metastasis tiles...
tiffload
tiffload
Extracting 4 of 4 val_metastasis tiles...
tiffload
Extracting 1 of 1 val_metastasis tiles...
tiffload
Extracting 8 of 8 val_metastasis tiles...
tiffload
tiffload
Extracting 13 of 13 val_metastasis tiles...
tiffload
Extracting 2 of 2 val_metastasis tiles...
tiffload
Extracting 3 of 3 val_metastasis tiles...
tiffload
Extracting 3 of 3 val_metastasis tiles...
tiffload
Extracting 2 of 2 val_metastasis tiles...
t